# Example Selctor (By MMR)

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. Example Selector (By MMR)

- SemanticSimilarity (의미유사도) : Cosine 유사도와 같이 벡터간 유사도 계산을 통한 측정값을 기준으로 유사도가 높은 것을 선택하는 알고리즘
    - 문제점 
        - 의미적으로 유사한 n개의 예제에 대해서, n개의 예제는 모두 유사성은 높음
        - 하지만 그 안에서 보다 다양한 예제들로 구성하도록 선택하지는 못함 (이럴 때 많이 사용함)
- MaxMarginalReleavance : 
    - 다양성과 관련성을 모두 고려하여 결과를 선택하는 데 사용
    - 결과의 중복성을 줄이면서도 가장 관련성 높은 정보를 제공하는 것 을 목표로 한 알고리즘임
    - 과정
        1. 가장 관련성 높은 항목 선택
        2. 이후 각 단계에서 현재 선택된 항목들과 관련성은 높으 면서, 가장 차별화된 항목을 찾아 선택 (lambda 값에 따라 조절)
            - lambda 값이 클수록 관련성을, 작을수록 다양성을 중시함

In [3]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
        """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
        """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
        """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
            중간 답변: 올드보이의 감독은 박찬욱입니다.
            추가 질문: 박찬욱은 어느 나라 출신인가요?
            중간 답변: 박찬욱은 대한민국 출신입니다.
            추가 질문: 기생충의 감독은 누구인가요?
            중간 답변: 기생충의 감독은 봉준호입니다.
            추가 질문: 봉준호는 어느 나라 출신인가요?
            중간 답변: 봉준호는 대한민국 출신입니다.
            최종 답변은: 예
        """,
    },
]

In [9]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=Chroma,
    k=1
)

In [10]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate.from_template(
    template="Question:\n{question}\nAnswer:\n{answer}\n",
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Qustion:\n{question}\nAnswer:",
    input_variables=["question"]
)

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

chain = prompt | llm

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
for chunk in chain.stream(question):
    print(chunk.content, end="", flush=True)

Number of requested results 20 is greater than number of elements in index 12, updating n_results = 12


이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
추가 질문: 1998년에 Bill Gates는 몇 살이었나요?
중간 답변: 1998년에는 Bill Gates가 43세였습니다.
최종 답변은: 43세